# Creating a spark session

In [1]:
# use tis command if you are using the jupyter notebook

import os
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").config(conf=SparkConf()).getOrCreate()

# loading the data and assigning the schema.

path_text_orders="file:///D://data-master/retail_db/orders"

orders_text=spark.read.format("text").load(path_text_orders)

orders_table=orders_text.selectExpr("cast(split(value,',') [0] as int) order_customer_id",
                                     "cast(split(value,',') [1] as date) order_date",
                                     "cast(split(value,',') [2] as int) order_id",
                                      "cast(split(value,',') [3] as string) order_status")

orders_table.show(2)

+-----------------+----------+--------+---------------+
|order_customer_id|order_date|order_id|   order_status|
+-----------------+----------+--------+---------------+
|                1|2013-07-25|   11599|         CLOSED|
|                2|2013-07-25|     256|PENDING_PAYMENT|
+-----------------+----------+--------+---------------+
only showing top 2 rows



# Launching in CLOUDERA vm 

In [ ]:
## to stop the warnings and info in saprk 1.6

sc.setLogLevel("ERROR")

In [ ]:
'''launch pysaprk'''

'''load the data'''

path_text_orders="/user/pruthviraj/sqoop_text/orders"

orders_text=sqlContext.read.format("text").load(path_text_orders)

orders_table=orders_text.selectExpr("cast(split(value,',') [0] as int) order_customer_id",
                                     "cast(split(value,',') [1] as date) order_date",
                                     "cast(split(value,',') [2] as int) order_id",
                                      "cast(split(value,',') [3] as string) order_status")

orders_table.count()

![tilte](https://pysparktutorials.files.wordpress.com/2018/05/load-vm.jpg)

##  once you load the data , these commands should work on both jupyter notebook and cloudera vms

## gruouping the data

#### groupBy is case sensitive , please make a note of it.

In [3]:
# Grouping the data and counting the elements

orders_table.groupBy("order_status").count().show()

+---------------+-----+
|   order_status|count|
+---------------+-----+
|PENDING_PAYMENT|15030|
|       COMPLETE|22899|
|        ON_HOLD| 3798|
| PAYMENT_REVIEW|  729|
|     PROCESSING| 8275|
|         CLOSED| 7556|
|SUSPECTED_FRAUD| 1558|
|        PENDING| 7610|
|       CANCELED| 1428|
+---------------+-----+



## aggretaing the data

#### to aggreagate the data we need to import the pyspark sql functions

In [13]:
# aggregating the data

from pyspark.sql import functions as f

orders_table.groupBy("order_status").agg(f.count(orders_table.order_status).alias("count"),\
                                         f.max(orders_table.order_id).alias("max")).show()

+---------------+-----+-----+
|   order_status|  cnt|  max|
+---------------+-----+-----+
|PENDING_PAYMENT|15030|12434|
|       COMPLETE|22899|12434|
|        ON_HOLD| 3798|12434|
| PAYMENT_REVIEW|  729|12433|
|     PROCESSING| 8275|12431|
|         CLOSED| 7556|12434|
|SUSPECTED_FRAUD| 1558|12429|
|        PENDING| 7610|12435|
|       CANCELED| 1428|12435|
+---------------+-----+-----+



# having cluase 

#### there is no having clause in the pyspark ,the best you can make use of filter

In [15]:
from pyspark.sql import functions as f

orders_table.groupBy("order_status").\
             agg(f.count(orders_table.order_status).alias("count"),\
             f.max(orders_table.order_id).alias("max")).\
             where("count > 5000").show()

+---------------+-----+-----+
|   order_status|count|  max|
+---------------+-----+-----+
|PENDING_PAYMENT|15030|12434|
|       COMPLETE|22899|12434|
|     PROCESSING| 8275|12431|
|         CLOSED| 7556|12434|
|        PENDING| 7610|12435|
+---------------+-----+-----+

